# Fee Splitter deployment
This script lays the foundations for autobribe by deploying the fee splitter while leaving the fee distribution unchanged (100% of the fees go to the fee collector).

To run this notebook you need to install the dependencies from the `scripts` poetry group as well
.

## Setup

In [9]:
import os

# importing scripting dependencies
import boa
import curve_dao
from eth_account import Account
from boa import NetworkEnv, Env


In [10]:
# use address book for relevant addresses
import tests.integration.address_book as ab

from dotenv import load_dotenv
load_dotenv()

# get env vars
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
PINATA_API_KEY = os.getenv("PINATA_API_KEY")
RPC_URL = os.environ.get("RPC_URL")
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")

## Select mode
Production mode cell is intentionally executed first to avoid deploying things by accident. If one runs all the cells sequentially things will be executed in forking mode.

To deploy in production mode, skip the execution of the fork mode cell.

In [11]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = NetworkEnv(RPC_URL)
boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)

TypeError: Cannot convert None of type <class 'NoneType'> to bytes

In [12]:
# fork mode (for testing) 
# we impersonate convex since they definitely have enough to push a vote
CONVEX_VOTERPROXY = "0x989aeb4d175e16225e39e87d0d97a3360524ad80"

forked_env = Env()
boa.set_env(forked_env)

boa.env.fork(RPC_URL)
boa.env.eoa = CONVEX_VOTERPROXY

## Compilation

In [13]:
# compiling contract (through boa's magic imports)
from contracts.fee_splitter import FeeSplitter

## Deployment

In [14]:
# only receiver for now is the fee collector
# we expect the current fee collection behavior to remain unchanged
receivers = [(ab.fee_collector, 10_000, False)]
fs = FeeSplitter(ab.crvusd, ab.controller_factory, receivers, ab.dao)

## Vote

In [17]:
ACTIONS = [
    (ab.controller_factory, "set_fee_receiver", fs)
]
DESCRIPTION = "Set the fee receiver to the fee splitter, to lay the foundation for autobribe, st-crvUSD and rebalancing donations. This vote **is not** about changing the fee distribution (100% of revenues will still go to veCRV holders)."
curve_dao.create_vote(ab.dao, ACTIONS, DESCRIPTION, ETHERSCAN_API_KEY, PINATA_API_KEY)

[15:16:53] INFO     Preparing EVM script                                                           ]8;id=556894;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/vote_utils.py\vote_utils.py]8;;\:]8;id=860239;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/vote_utils.py#43\43]8;;\

           INFO     EVM script generated:                                                          ]8;id=43960;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/vote_utils.py\vote_utils.py]8;;\:]8;id=923648;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/vote_utils.py#69\69]8;;\
                    b"\x00\x00\x00\x01@\x90u@\xd8\xa6\xc6\\cw\x85\xe8\xf8\xb7B\xaek\x0b\x99h\x00\x                 
                    00\x00\xc4\xb6\x1d'\xf6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc93/\                 
                    xdc\xb1\xc4\x91\xdc\xc6\x83\xba\xe8o\xe3\xcbp6\x078\xbc\x00\x00\x00\x00\x00\x0                 
                    0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\                 
                    x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0                 
                    0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00                 
                    \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x                 
                    00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00$\xe4\x1a\xb7q\x00\x00\x00\x00\x00\x                 
                    00\x00\x00\x00\x00\x00\x00\xe7l=\xd7:\xa9\x18N\xe7\x81\xc7fA\xe2\xfc5\x9fA\xdf                 
                    E\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\                 
                    x00\x00\x00\x00\x00\x00\x00\x00\x00"                                                           

[15:16:55] INFO     Pinned Vote description to                                                           ]8;id=908311;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/ipfs.py\ipfs.py]8;;\:]8;id=380140;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/ipfs.py#39\39]8;;\
                    ipfs:bafkreigov6ku34xskmltm6v27unxn7msml4mqmbculcqtn5sol5epdwaxq                               

[15:16:56] ERROR    EOA cannot create new vote. Either there isn't enough veCRV balance orEOA        ]8;id=573931;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/__init__.py\__init__.py]8;;\:]8;id=259033;file:///Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_dao/__init__.py#42\42]8;;\
                    created a vote less than 12 hours ago.                                                         
                    ╭───────────────────── Traceback (most recent call last) ──────────────────────╮               
                    │ /Users/albe/Repos/curve/autobribe/.venv/lib/python3.12/site-packages/curve_d │               
                    │ ao/__init__.py:40 in create_vote                                             │               
                    │                                                                              │               
                    │   37 │   )                                                                   │               
                    │   38 │                                                                       │               
                    │   39 │   try:                                                                │               
                    │ ❱ 40 │   │   assert voting.canCreateNewVote(boa.env.eoa)                     │               
                    │   41 │   except Exception:                                                   │               
                    │   42 │   │   logger.exception(                                               │               
                    │   43 │   │   │   "EOA cannot create new vote. Either there isn't enough veCR │               
                    ╰──────────────────────────────────────────────────────────────────────────────╯               
                    AssertionError                                                                                 

BoaError:   (<AragonVoting interface at 0xE478de485ad2fe566d49342Cbd03E49ed7DB3356>.newVote(bytes,string,bool,bool) -> ['uint256'])

   <Unknown contract 0xa4d1a2693589840babb7f3a44d14fdf41b3bf1fe>

  (<AragonVoting interface at 0xE478de485ad2fe566d49342Cbd03E49ed7DB3356>.newVote(bytes,string,bool,bool) -> ['uint256'])